In [1]:
from ortools.linear_solver import pywraplp
import numpy as np

In [2]:
# Определяем переменные

# Число возможностей
num_opportunities = 6

# 1 - фоткаем, - 0 сбрасываем
c = [1, 1, 0, 0, 0, 0]  # Priority weights

# Штраф за то что храним данные на спутнике
d = np.ones(num_opportunities)

# Скорость 4 Гб/c - фоткаем, 1 Гб/c - передаем. а - это время возможности * скорость отправки или передачи
a = [10*1000, 10*1000, 6*1000, 4*1000, 1*1000, 5*1000]

# Это сеты в которых записано то что мы делаем
s_img =  {0,1}
s_dl = {2, 3, 4, 5}

s_mutex = [[4,3]]
# s_mutex = []

op_sat_id = [0,1,1,1,0,0]
op_sat_id_dict = {0:[0,4,5],
                  1:[1,2,3]}
n_sat = 2
alpha = 0.000001

# capacity
cap = 20 * 1000

In [3]:
from itertools import combinations

def get_pairs(s):
    return list(combinations(s, 2))
get_pairs(s_mutex[0])

[(4, 3)]

In [4]:
def prev_con_id(con_id):
  prev_con = None
  sat_key = op_sat_id[con_id]
  for el in op_sat_id_dict[sat_key]:
    if el == con_id:
      return prev_con
    prev_con = el
print(prev_con_id(1))

None


In [5]:
def cond_for_moment_i(con_id):
  prev_con = None
  res = {}
  for k in op_sat_id_dict:
    for el in op_sat_id_dict[k]:
      if el > con_id:
        break
      res[k] = el
      prev_con = el
  return res

print(cond_for_moment_i(5).values())

dict_values([5, 3])


In [6]:
def foo(c_coefs, d_coefs, a_coefs, s_img, s_dl, s_mutex,op_sat_id, op_sat_id_dict, n_sat, alpha, cap ):
  # Создаем линейную модель
  solver = pywraplp.Solver('Satellite', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  x = [solver.IntVar(0, 1, f'x{i}') for i in range(num_opportunities)]
  y = [solver.NumVar(0, cap, f'y{i}') for i in range(num_opportunities)]

  # Определяем целевую функцию

  objective = solver.Objective()


  for el in s_mutex:
    for l,r in get_pairs(el):
      solver.Add(x[l] + x[r] <= 1)


  # Определяем ограничения
  for i in range(num_opportunities):
    solver.Add(x[i] <= y[i])
    pr_con_id = prev_con_id(i)
    tmp = 0
    if pr_con_id is not None:
      tmp = y[pr_con_id]
    if i in s_img:
        solver.Add(y[i] == tmp + a[i] * x[i])
    if i in s_dl:
        solver.Add(y[i] >= tmp - a[i] * x[i])

  # for i in range(num_opportunities):
  #     objective.SetCoefficient(x[i], -c[i])
  #     print(cond_for_moment_i(i))
  #     for k in cond_for_moment_i(i).values():
  #       objective.SetCoefficient(y[k], alpha * d[i])
  # objective.SetMinimization()


  res = 0
  for i in range(num_opportunities):
    res -= x[i]* c[i]
    for k in cond_for_moment_i(i).values():
      res +=  y[k]*alpha*d[k]
  solver.Minimize(res)
  # Решаем задачу
  status = solver.Solve()

  # Выводим результаты
  if status == pywraplp.Solver.OPTIMAL:
      print('Решение найдено')
      print(f'Сумма приоритетов: {objective.Value()}')
      for i in range(num_opportunities):
          s = sum([round(y[k].solution_value()) for k in cond_for_moment_i(i).values()])
          print(f'x{i}: {x[i].solution_value()}, y{i}: {round(y[i].solution_value())}, sat # {op_sat_id[i]}, sum={s}')
  else:
      print('Решение не найдено')

In [7]:
foo(c,d,a,s_img, s_dl,s_mutex,op_sat_id, op_sat_id_dict, n_sat, alpha,cap)

Решение найдено
Сумма приоритетов: -1.930997
x0: 1.0, y0: 10000, sat # 0, sum=10000
x1: 1.0, y1: 10000, sat # 1, sum=20000
x2: 1.0, y2: 4000, sat # 1, sum=14000
x3: 1.0, y3: 1, sat # 1, sum=10001
x4: 0.0, y4: 10000, sat # 0, sum=10001
x5: 1.0, y5: 5000, sat # 0, sum=5001


### Одна съемка, 3 передачи

In [8]:
# Число возможностей
num_opportunities = 4

# 1 - фоткаем, - 0 сбрасываем
c = [1, 0, 0, 0]  # Priority weights

# Штраф за то что храним данные на спутнике
d = np.ones(num_opportunities)

# Скорость 4 Гб/c - фоткаем, 1 Гб/c - передаем. а - это время возможности * скорость отправки или передачи
a = [100*1000, 1*1000, 5*1000, 5*1000]

# Это сеты в которых записано то что мы делаем
s_img =  {0}
s_dl = {1,2, 3}

s_mutex = []

op_sat_id = [0,0,0,0]
op_sat_id_dict = {0:[0,1,2,3]}
n_sat = 1
alpha = 0.000001

# capacity
cap = 2000 * 1000

In [9]:
foo(c,d,a,s_img, s_dl,s_mutex,op_sat_id, op_sat_id_dict, n_sat, alpha,cap)

Решение найдено
Сумма приоритетов: -0.6180000000000001
x0: 1.0, y0: 100000, sat # 0, sum=100000
x1: 1.0, y1: 99000, sat # 0, sum=99000
x2: 1.0, y2: 94000, sat # 0, sum=94000
x3: 1.0, y3: 89000, sat # 0, sum=89000
